### 얼굴 인식


In [ ]:
import cv2

#얼굴과 눈을 감지하는 미리 학습된 Haar cascades를 로드. 이것들은 감지할 대상의 특징을 포함하는 XML 파일
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

#기본 카메라(0)를 위한 비디오 캡처 객체를 초기화
video_capture = cv2.VideoCapture(0)


while True:
    #ret:프레임이 성공적으로 읽혔는지 나타내는 변수 , frame: 이미지 데이터가 포함된 배열 
    ret, frame = video_capture.read()
    #컬러 -> 흑백
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detectMultiScale 사용하여 흑백이미지에서 얼굴 감지, 얼굴 둘러싼 사각형 리스트 반환
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
    )

    #얼굴 주변 사각형 그림
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    #frame 출력
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [2]:
pip install cvlib

     |████████████████████████████████| 13.1 MB 8.7 MB/s eta 0:00:01
  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046371 sha256=b51537610cfc752f59f227b074558eb2b4e050a61ed67b02bbb77964313e0635
  Stored in directory: /Users/hwangseoyeon/Library/Caches/pip/wheels/bb/11/55/82ced947752cc4056335a4ee9c304a93d43ce857230538f0ae
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12074 sha256=6eff0ec815ed3055c54d56f644609d6dd45a5daaacfb496818936f5ab240f350
  Stored in directory: /Users/hwangseoyeon/Library/Caches/pip/wheels/2c/67/ed/d84123843c937d7e7f5ba88a270d11036473144143355e2747
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=c7ac1817aec2bf173179f7329131a43a6d1c9e4477d44ec27b2569c44008d615
  Stored in directory: /Users/hwangseoyeon/Library/Caches/pip/wheels/59/1b/52/0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89
Successfully built cvlib progressbar imutils
Note: you may need to restart the kern

In [3]:
pip install --upgrade cvlib

Requirement already up-to-date: cvlib in /Users/hwangseoyeon/opt/anaconda3/lib/python3.8/site-packages (0.2.7)
Note: you may need to restart the kernel to use updated packages.


In [7]:
!if not exist "./files" mkdir files
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

zsh:1: parse error near `files'
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file 
  1  908k    1 16384    0     0  54979      0  0:00:16 --:--:--  0:00:16 55164
curl: (23) Failure writing output to destination
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file ./files/emotion_model.hdf5: No such file or 
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (23) Failure writing output to destination


In [9]:
!mkdir files
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73698    0 73698    0     0   9192      0 --:--:--  0:00:08 --:--:-- 19338


### 표정 인식

In [ ]:
import cv2
import numpy as np
import cvlib as cv
from keras.models import load_model
from tensorflow.keras.utils import img_to_array,load_img

# 감정 분류를 위한 사전 학습된 딥러닝 모델을 로드
emotion_classifier = load_model('emotion_model.h5', compile=False)

# 성별 분류를 위한 사전 학습된 딥러닝 모델을 로드
gender_classifier = load_model('gender_model.h5', compile=False)

#감정 분류 모델의 출력 클래스에 해당하는 감정 목록을 초기화
EMOTIONS = ["Angry", "Disgusting", "Fearful", "Happy", "Sad", "Surpring", "Neutral"]

# 성별 분류 모델의 출력 클래스에 해당하는 감정 목록을 초기화
GENDER = ['Female','Male']

#Haar Cascade 분류기를 사용하여 얼굴 감지 분류기를 초기화
face_detection = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


cap = cv2.VideoCapture(0)

while True:
    # 카메라 프레임 읽기
    ret, frame = cap.read()

    # 프레임을 회색으로 전환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #이미지에서 얼굴 감지, detectMultiScale: 이미지에서 감지된 얼굴을 나타내는 사각형 목록 반환
    faces = face_detection.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # for each face detected, recognize the emotion
    for (x, y, w, h) in faces:
        
        #현재 얼굴 사각형의 좌표 사용하여 ROI 추출
        roi_gray = gray[y:y+h, x:x+w]
        #ROI 이미지 크기 64,64 조정 -> 전처리
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi = roi_gray.astype("float") / 255.0
        #ROI 이미지를 numpy배열로 변환
        roi = img_to_array(roi)
        #배열에 차원 추가
        roi = np.expand_dims(roi, axis=0)
        
        #이미 학습된 모델로 감정 예측
        preds_1 = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds_1)
        label_1 = EMOTIONS[preds_1.argmax()]
        
        #성별
        roi_1 = np.concatenate([roi]*3, axis=-1)
        preds =gender_classifier.predict(roi_1)[0]
        print(preds)
        if preds < 0.7 :
            gender_label = 'Female'
        else :
            gender_label = 'Male'
        #gender_probability = np.max(preds)
        #gender_index = preds[0].argmax()
        #gender_label = GENDER[gender_index]
        #gender_confidence = gender_preds[0][gender_index]

        # 표정 라벨과, 얼굴 주위에 경계상자 표시 
        cv2.putText(frame, label_1, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        cv2.putText(frame, gender_label, (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    cv2.imshow('Emotion', frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 148ms/step
[0.72847646]
1/1 [==============================] - 0s 20ms/step
[0.9295184]
1/1 [==============================] - 0s 21ms/step
[0.92830944]
1/1 [==============================] - 0s 19ms/step
[0.9469611]
1/1 [==============================] - 0s 20ms/step
[0.93775374]
1/1 [==============================] - 0s 19ms/step
[0.9179467]
1/1 [==============================] - 0s 19ms/step
[0.8869049]
1/1 [==============================] - 0s 20ms/step
[0.8747]
1/1 [==============================] - 0s 19ms/step
[0.8935397]
1/1 [==============================] - 0s 20ms/step
[0.9432874]
1/1 [==============================] - 0s 19ms/step
[0.94200516]
1/1 [==============================] - 0s 20ms/step
[0.9405547]
1/1 [==============================] - 0s 18ms/step
[0.9405254]
1/1 [==============================] - 0s 19ms/step
[0.9481104]
1/1 [==============================] - 0s 19ms/step
[0.946618]
1/1 [==============================] - 

1/1 [==============================] - 0s 20ms/step
[0.70227224]
1/1 [==============================] - 0s 22ms/step
[0.64172345]
1/1 [==============================] - 0s 19ms/step
[0.7477948]
1/1 [==============================] - 0s 20ms/step
[0.5784587]
1/1 [==============================] - 0s 23ms/step
[0.6999387]
1/1 [==============================] - 0s 19ms/step
[0.59565765]
1/1 [==============================] - 0s 19ms/step
[0.7179294]
1/1 [==============================] - 0s 24ms/step
[0.7090424]
1/1 [==============================] - 0s 19ms/step
[0.5995824]
1/1 [==============================] - 0s 20ms/step
[0.62883097]
1/1 [==============================] - 0s 21ms/step
[0.70008767]
1/1 [==============================] - 0s 20ms/step
[0.68579054]
1/1 [==============================] - 0s 19ms/step
[0.68995994]
1/1 [==============================] - 0s 18ms/step
[0.6488568]
1/1 [==============================] - 0s 19ms/step
[0.6555148]
1/1 [============================

1/1 [==============================] - 0s 19ms/step
[0.55809104]
1/1 [==============================] - 0s 20ms/step
[0.53034556]
1/1 [==============================] - 0s 19ms/step
[0.55707383]
1/1 [==============================] - 0s 19ms/step
[0.581582]
1/1 [==============================] - 0s 19ms/step
[0.8592317]
1/1 [==============================] - 0s 19ms/step
[0.64145505]
1/1 [==============================] - 0s 20ms/step
[0.46213287]
1/1 [==============================] - 0s 19ms/step
[0.67710835]
1/1 [==============================] - 0s 19ms/step
[0.82090384]
1/1 [==============================] - 0s 18ms/step
[0.76831365]
1/1 [==============================] - 0s 18ms/step
[0.81431764]
1/1 [==============================] - 0s 19ms/step
[0.64854217]
1/1 [==============================] - 0s 18ms/step
[0.5549004]
1/1 [==============================] - 0s 19ms/step
[0.6878146]
1/1 [==============================] - 0s 19ms/step
[0.68252707]
1/1 [=========================

1/1 [==============================] - 0s 20ms/step
[0.7005523]
1/1 [==============================] - 0s 19ms/step
[0.672661]
1/1 [==============================] - 0s 19ms/step
[0.71881455]
1/1 [==============================] - 0s 20ms/step
[0.6905272]
1/1 [==============================] - 0s 19ms/step
[0.64326936]
1/1 [==============================] - 0s 19ms/step
[0.5830642]
1/1 [==============================] - 0s 19ms/step
[0.46677274]
1/1 [==============================] - 0s 20ms/step
[0.7031006]
1/1 [==============================] - 0s 21ms/step
[0.8296761]
1/1 [==============================] - 0s 20ms/step
[0.6011828]
1/1 [==============================] - 0s 19ms/step
[0.67404425]
1/1 [==============================] - 0s 20ms/step
[0.9280854]
1/1 [==============================] - 0s 19ms/step
[0.9485142]
1/1 [==============================] - 0s 20ms/step
[0.6486412]
1/1 [==============================] - 0s 20ms/step
[0.57234985]
1/1 [==============================]